# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from citipy import citipy
from pprint import pprint

# Import API key and configure gmaps
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df=pd.read_csv("Results/Cities.csv")
cities_df

,City Number,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,1.0,Nur-Sultan,0.0,1.587141e+09,65.0,51.18,71.45,44.60,15.01027
1,2.0,Tezu,45.0,1.587142e+09,66.0,27.92,96.17,69.87,6.30834
2,3.0,Bluff,98.0,1.587142e+09,94.0,-46.60,168.33,46.99,4.45163
3,4.0,Vaini,20.0,1.587142e+09,83.0,-21.20,-175.20,75.20,13.01934
4,5.0,Mahébourg,40.0,1.587142e+09,69.0,-20.41,57.70,77.00,33.53263
...,...,...,...,...,...,...,...,...,...
582,583.0,Crossville,1.0,1.587142e+09,33.0,35.95,-85.03,66.00,33.53263
583,584.0,Buta,81.0,1.587142e+09,71.0,2.79,24.73,78.39,9.79806
584,585.0,Mount Isa,16.0,1.587141e+09,29.0,-20.73,139.50,75.20,15.50241
585,586.0,Kaeo,90.0,1.587142e+09,92.0,-35.10,173.78,61.00,6.71100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities_df[["Lat", "Lng"]]
humidity= cities_df["Humidity"]

#Display map

fig=gmaps.figure()

#Alternative to make a bigger figure
# figure_layout={
#     "width": "400px",
#     "height": "300px",
#     "border": "1px solid black",
#     "padding": "1px",
#     "margin": "0 auto 0 auto"}
# fig= gmaps.figure(layout= figure_layout)



# Create heat layer and add it to figure
heat_layer=gmaps.heatmap_layer(locations=locations, weights=humidity, \
                               dissipating=True, max_intensity=100)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
best_cities_df=cities_df.loc[(cities_df["Max Temp"] <=80) & (cities_df["Max Temp"] >=70) &\
                         (cities_df["Cloudiness"] < 20) & (cities_df["Wind Speed"] <15)]
best_cities_df= best_cities_df.reset_index(drop=True)
best_cities_df

,City Number,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,19.0,Castro,5.0,1.587141e+09,26.0,-24.79,-50.01,73.47,3.75816
1,82.0,Makkah al Mukarramah,0.0,1.587141e+09,41.0,21.43,39.83,75.58,1.25272
2,104.0,Sawākin,0.0,1.587142e+09,60.0,19.11,37.33,77.14,4.04897
3,197.0,Kīhei,1.0,1.587141e+09,72.0,20.79,-156.47,70.00,10.51390
4,269.0,Lahaina,1.0,1.587142e+09,72.0,20.88,-156.68,70.00,10.51390
5,293.0,Cabo San Lucas,5.0,1.587142e+09,53.0,22.89,-109.91,77.00,10.51390
6,398.0,Sipacapa,1.0,1.587142e+09,42.0,15.20,-91.67,71.01,6.71100
7,415.0,Buin,3.0,1.587142e+09,33.0,-33.73,-70.75,73.40,10.51390
8,435.0,Kariba,13.0,1.587142e+09,53.0,-16.52,28.80,76.98,4.09371
9,476.0,Pontes e Lacerda,18.0,1.587142e+09,49.0,-15.23,-59.34,78.89,14.80894


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df= pd.DataFrame({
    "Hotel Name":[],
    "City":[],
    "Country":[],
    "Lat":[],
    "Lng":[]
})
hotel_df

,Hotel Name,City,Country,Lat,Lng


In [6]:
#Generate base query_url and parameters 
query_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params={
    "location":"",
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key,
}

#Iterate through each row of best_cities dataframe and generate API calls
for index, row in best_cities_df.iterrows():
    name= row["City"]
    coordinates=(f'{row["Lat"]}, {row["Lng"]}')
    params["location"]= coordinates
    print(f'Retrieving results for index {index}: {name}')
    response=requests.get(query_url, params=params).json()
    results=response["results"]
    
    #Populate the new hotel_df with retrieved information
    try:
        print(f'The first hotel nearby {name} is {results[0]["name"]}')
        hotel_df.loc[index, "Hotel Name"]= results[0]["name"]
        hotel_df.loc[index, "Lat"]= row["Lat"]
        hotel_df.loc[index, "Lng"]= row["Lng"]
        city=citipy.nearest_city(row["Lat"], row["Lng"])
        hotel_df.loc[index, "City"]=city.city_name.capitalize()
        hotel_df.loc[index, "Country"]= city.country_code.upper()
        
    except (KeyError, IndexError):
        print(f'----Missing field/result for {name}. Skipping----')

hotel_df

Retrieving results for index 0: Castro
The first hotel nearby Castro is Hotel Buganville Express
Retrieving results for index 1: Makkah al Mukarramah
The first hotel nearby Makkah al Mukarramah is Hilton Makkah Convention Hotel
Retrieving results for index 2: Sawākin
The first hotel nearby Sawākin is اولاد خضر
Retrieving results for index 3: Kīhei
The first hotel nearby Kīhei is Kohea Kai Maui, Ascend Hotel Collection
Retrieving results for index 4: Lahaina
The first hotel nearby Lahaina is Ka'anapali Beach Hotel
Retrieving results for index 5: Cabo San Lucas
The first hotel nearby Cabo San Lucas is Villa Del Palmar Beach Resort & Spa Cabo San Lucas
Retrieving results for index 6: Sipacapa
----Missing field/result for Sipacapa. Skipping----
Retrieving results for index 7: Buin
The first hotel nearby Buin is Motel INSIDE
Retrieving results for index 8: Kariba
The first hotel nearby Kariba is Caribbea Bay Hotel
Retrieving results for index 9: Pontes e Lacerda
The first hotel nearby Ponte

,Hotel Name,City,Country,Lat,Lng
0,Hotel Buganville Express,Castro,BR,-24.79,-50.01
1,Hilton Makkah Convention Hotel,Mecca,SA,21.43,39.83
2,اولاد خضر,Sawakin,SD,19.11,37.33
3,"Kohea Kai Maui, Ascend Hotel Collection",Kihei,US,20.79,-156.47
4,Ka'anapali Beach Hotel,Lahaina,US,20.88,-156.68
5,Villa Del Palmar Beach Resort & Spa Cabo San L...,Cabo san lucas,MX,22.89,-109.91
7,Motel INSIDE,Buin,CL,-33.73,-70.75
8,Caribbea Bay Hotel,Kariba,ZW,-16.52,28.80
9,Plaza Hotel,Pontes e lacerda,BR,-15.23,-59.34
10,Hotel Fanilo,Vangaindrano,MG,-23.35,47.60


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))